# Conectando OpenAI con python a BBDD (págila): Extracción de metadatos

In [1]:
from utils import get_embedding, full_md_extraction_pipeline, simple_md_extraction_pipeline, search_most_similar_md, conect_to_bbdd, close_conection
from utils import store_metadata_df_to_pickle, enviar_promt_chat_completions_mode
from calcular_tokens import num_tokens_from_string, num_tokens_from_messages

import pandas as pd
import openai
import numpy as np
import os

In [2]:
from dotenv import load_dotenv

# Cargar variables de entorno desde .env
load_dotenv()

# Acceder a la API key
api_key = os.getenv("API_KEY")
openai.api_key = api_key

In [3]:
# mi_prompt = [
#     {'role': 'system',    'content':'eres un asistente agradable y gracioso'},               
#     {'role': 'user',      'content':'hola que tal estas'},
#     #{'role': 'assistant', 'content': 'Devuélveme la respuesta en este formato: \n```sql\nEL CODIGO SQL;\n```'}
# ]

# respuesta = enviar_promt_chat_completions_mode(mensaje=mi_prompt)

# print(respuesta)

## Carga de metadatos de la BBDD

Si el archivo.pickle existe en el directorio con los metadatos nos dice que ya existe, de lo contrario, los extrae de la bbdd. LA BASE DE DATOS DEBE ESTAR LEVANTADA 

In [4]:
pagila = {
    'bbdd_name':'pagila', 
    'user'     :'postgres', 
    'password' :'123', 
    'host'     :'localhost', 
    'port'     :'5432'
}

store_metadata_df_to_pickle(**pagila)

El archivo ya existe: data/pagila_simple_metadata.pickle
El archivo ya existe: data/pagila_full_metadata.pickle


'Proceso de almacenamiento de metadatos para pagila completado.'

In [5]:
rna_central = {
    'bbdd_name': 'pfmegrnargs', 
    'user'     : 'reader', 
    'password' : 'NWDMCE5xdipIjRrp', 
    'host'     : 'hh-pgsql-public.ebi.ac.uk', 
    'port'     : '5432'
}

store_metadata_df_to_pickle(**rna_central)

El archivo ya existe: data/pfmegrnargs_simple_metadata.pickle
El archivo ya existe: data/pfmegrnargs_full_metadata.pickle


'Proceso de almacenamiento de metadatos para pfmegrnargs completado.'

In [6]:
simple_md_df = pd.read_pickle('data/pagila_simple_metadata.pickle')
full_md_df = pd.read_pickle('data/pagila_full_metadata.pickle')

display(simple_md_df.head())
display(full_md_df.head())

,metadata_str,md_str_tokens,embedding
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,"[-0.010325274430215359, 0.012428195215761662, ..."
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,"[-0.029717877507209778, 0.01876918599009514, 0..."
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,"[-0.03288605436682701, -0.005693415179848671, ..."
store,"{""store"": [""store_id"", ""manager_staff_id"", ""ad...",20,"[-0.029268378391861916, 0.008447147905826569, ..."
address,"{""address"": [""address_id"", ""address"", ""address...",32,"[-0.027424100786447525, 0.024846235290169716, ..."


,metadata_str,md_str_tokens,embedding
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,"[-0.0006032844539731741, 0.024174265563488007,..."
language,"table_name:language\n{""col_name"": ""language_id...",103,"[0.0010160236852243543, 0.02792864479124546, 0..."
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,"[-0.0034067181404680014, 0.005942848976701498,..."
store,"table_name:store\n{""col_name"": ""store_id"", ""ty...",139,"[-0.006352687254548073, 0.021349918097257614, ..."
address,"table_name:address\n{""col_name"": ""address_id"",...",271,"[-0.0030516614206135273, 0.02602650411427021, ..."


## Extraemos los metadatos más similares a una consulta del usuario

La funcion search most similar metadata tiene dos limitantes el numero de tablas minimo que y el numero maximo de tokens 

In [9]:
consulta = '''
dame los nombres de los actores y actrices que hayan participado en 
peliculas con una duracion superior 180 minutos. incluye tambien la 
duración exacta de la pelicula
'''

token_limit = 1000

most_similar_md_simple = search_most_similar_md(prompt= consulta,metadata=simple_md_df, n_resultados=14, lim_tokens=token_limit)
most_similar_md_full = search_most_similar_md(prompt= consulta,metadata=full_md_df, n_resultados=14, lim_tokens=token_limit )

In [10]:
display(most_similar_md_simple)
display(most_similar_md_full)

,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
film_actor,"{""film_actor"": [""actor_id"", ""film_id"", ""last_u...",16,0.733055,"[-0.033447034657001495, 0.003196529345586896, ...",16
film,"{""film"": [""film_id"", ""title"", ""description"", ""...",53,0.732425,"[-0.010325274430215359, 0.012428195215761662, ...",69
actor,"{""actor"": [""actor_id"", ""first_name"", ""last_nam...",19,0.725807,"[-0.03288605436682701, -0.005693415179848671, ...",88
film_category,"{""film_category"": [""film_id"", ""category_id"", ""...",16,0.700418,"[-0.025663767009973526, 0.014611182734370232, ...",104
inventory,"{""inventory"": [""inventory_id"", ""film_id"", ""sto...",19,0.692363,"[-0.01771996170282364, 0.002168742474168539, 0...",123
language,"{""language"": [""language_id"", ""name"", ""last_upd...",14,0.686968,"[-0.029717877507209778, 0.01876918599009514, 0...",137
rental,"{""rental"": [""rental_id"", ""rental_date"", ""inven...",34,0.684099,"[-0.025464307516813278, 0.0028804554603993893,...",171
staff,"{""staff"": [""staff_id"", ""first_name"", ""last_nam...",42,0.676507,"[-0.018970182165503502, 0.008695820346474648, ...",213
category,"{""category"": [""category_id"", ""name"", ""last_upd...",14,0.674761,"[-0.030182400718331337, 0.017262794077396393, ...",227
address,"{""address"": [""address_id"", ""address"", ""address...",32,0.674351,"[-0.027424100786447525, 0.024846235290169716, ...",259


,metadata_str,md_str_tokens,similarity,embedding,token_cumsum
actor,"table_name:actor\n{""col_name"": ""actor_id"", ""ty...",138,0.742399,"[-0.0034067181404680014, 0.005942848976701498,...",138
film_actor,"table_name:film_actor\n{""col_name"": ""actor_id""...",107,0.740795,"[-0.00541864987462759, 0.010073642246425152, 0...",245
film,"table_name:film\n{""col_name"": ""film_id"", ""type...",443,0.723773,"[-0.0006032844539731741, 0.024174265563488007,...",688
film_category,"table_name:film_category\n{""col_name"": ""film_i...",107,0.718325,"[0.0017659964505583048, 0.02148573473095894, -...",795
inventory,"table_name:inventory\n{""col_name"": ""inventory_...",138,0.709392,"[0.00015317850920837373, 0.015428561717271805,...",933


In [11]:
print(len(most_similar_md_simple))
print(len(most_similar_md_full))

14
5


In [12]:
from utils import get_string_from_metadata_df

simple_md_str = get_string_from_metadata_df(most_similar_md_simple)
full_md_str = get_string_from_metadata_df(most_similar_md_full)

print('SIMPLE METADATA')
print(simple_md_str)
print()
print('FULL METADATA')
print(full_md_str)

SIMPLE METADATA
{"film_actor": ["actor_id", "film_id", "last_update"]}
{"film": ["film_id", "title", "description", "release_year", "language_id", "rental_duration", "rental_rate", "length", "replacement_cost", "rating", "last_update", "special_features", "fulltext"]}
{"actor": ["actor_id", "first_name", "last_name", "last_update"]}
{"film_category": ["film_id", "category_id", "last_update"]}
{"inventory": ["inventory_id", "film_id", "store_id", "last_update"]}
{"language": ["language_id", "name", "last_update"]}
{"rental": ["rental_id", "rental_date", "inventory_id", "customer_id", "return_date", "staff_id", "last_update"]}
{"staff": ["staff_id", "first_name", "last_name", "address_id", "email", "store_id", "active", "username", "password", "last_update", "picture"]}
{"category": ["category_id", "name", "last_update"]}
{"address": ["address_id", "address", "address2", "district", "city_id", "postal_code", "phone", "last_update"]}
{"city": ["city_id", "city", "country_id", "last_update

## Todo el pipeline en una función

In [1]:
from utils import nlp_to_BBBD_with_metadata_to_text_df_2

consulta_nlp = input('dame tu consutla: ')
print(consulta_nlp)

# consulta_nlp=  '''
# dame los nombres de los actores y actrices que hayan participado en 
# peliculas con una duracion superior 180 minutos. incluye tambien la 
# duración exacta de la pelicula
# '''
token_limit= 1000

tabla_texto= nlp_to_BBBD_with_metadata_to_text_df_2(
    consulta_nlp=consulta_nlp, 
    bbdd_name='pagila', 
    user= 'postgres', 
    password='123', 
    metadata_mode='full', 
    n_tablas= 10, 
    max_tokens=token_limit)

print(tabla_texto)

dame los actores con nombres de la a a la h ordenados alfabeticamente
El archivo ya existe: data/pagila_simple_metadata.pickle
El archivo ya existe: data/pagila_full_metadata.pickle
|    |   actor_id | first_name   | last_name   | last_update                |
|---:|-----------:|:-------------|:------------|:---------------------------|
|  0 |         71 | Adam         | Grant       | 2013-05-26 14:47:57.620000 |
|  1 |        132 | Adam         | Hopper      | 2013-05-26 14:47:57.620000 |
|  2 |        165 | Al           | Garland     | 2013-05-26 14:47:57.620000 |
|  3 |        173 | Alan         | Dreyfuss    | 2013-05-26 14:47:57.620000 |
|  4 |        146 | Albert       | Johansson   | 2013-05-26 14:47:57.620000 |
|  5 |        125 | Albert       | Nolte       | 2013-05-26 14:47:57.620000 |
|  6 |         29 | Alec         | Wayne       | 2013-05-26 14:47:57.620000 |
|  7 |         65 | Angela       | Hudson      | 2013-05-26 14:47:57.620000 |
|  8 |        144 | Angela       | Wit